In [1]:
%matplotlib widget
import os
import shutil
import getpass
import subprocess

from nilearn_tools import compute_connectome, subjectList
from acme import ParallelMap, esi_cluster_setup

In [2]:
saveto="neuro-bsp"

outfolder = os.path.join(os.path.abspath(os.path.expanduser(f"/cs/home/{getpass.getuser()}")), saveto)
shutil.rmtree(outfolder, ignore_errors=True)
os.makedirs(outfolder, exist_ok=True)

In [3]:
myClient = esi_cluster_setup(n_workers=10, partition="8GBXS")

/opt/conda/envs/esi-2023a/lib/python3.10/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37981 instead
  warnings.warn(
<slurm_cluster_setup> SLURM workers ready: 10/10 	[elapsed time 00:13 | timeout at 01:00]
ACME - INFO -  Parallel computing client ready, dashboard accessible at http://10.100.32.17:37981/status


In [8]:
with ParallelMap(compute_connectome, subjectList, outfolder, output_dir=outfolder, result_shape=(264, 264, None)) as pmap:
    results = pmap.compute()

ACME > ANNOUNCE <  This is ACME v. 2023.4
ACME - INFO -  Attaching to parallel computing client <Client: 'tcp://10.100.32.17:32851' processes=10 threads=10, memory=74.50 GiB>
ACME - INFO -  Preparing 21 parallel calls of `compute_connectome` using 10 workers
100% |██████████| 21/21 [02:00<00:00]
ACME > ANNOUNCE <  SUCCESS!
ACME - INFO -  Results have been saved to /cs/home/fuertingers/neuro-bsp/compute_connectome.h5


In [11]:
imgs = os.path.join(outfolder, "*.png")
gif = os.path.join(outfolder, f"{saveto}.gif")
subprocess.run(f"convert -size 1280x960 -delay 0 -loop 0 $(ls {imgs} | sort -V) {gif}",
               text=True, shell=True, check=True,
               stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

CompletedProcess(args='convert -size 1280x960 -delay 0 -loop 0 $(ls /cs/home/fuertingers/neuro-bsp/*.png | sort -V) /cs/home/fuertingers/neuro-bsp/neuro-bsp.gif', returncode=0, stdout='')

In [12]:
shutil.copy(gif, os.path.abspath(os.path.expanduser("~")))

'/gs/home/fuertingers/neuro-bsp.gif'